In [41]:
SHOW_OUTPUTS = True

def ShowTableau(c, a, b, certificate, optimal):
    print("\n")
    print("c[]: {}".format(c))
    print("A[] : {}".format(a))
    print("b[] : {}".format(b))
    print("Optimal : {}".format(optimal))
    print("\n")

def Identity(n):
    m=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        m[i][i] = 1
    return m

def Pivot(which_row, which_column, c, a, b, certificate, operations, optimal):
    
    pivot = a[which_row][which_column]
    if(SHOW_OUTPUTS): print("Pivot: {}".format(pivot))
    
    if(SHOW_OUTPUTS): print("Iterate A[] over the pivot: {}".format(pivot))
    for i in range(len(a[which_row])):
        if(a[which_row][i] != 0):
            a[which_row][i] = a[which_row][i]/pivot
            
    if(SHOW_OUTPUTS): print("Iterate operations[] over the pivot: {}".format(pivot))
    for i in range(len(a[which_row])):
        if(operations[which_row][i] != 0):
            operations[which_row][i] = operations[which_row][i]/pivot
    
    if(SHOW_OUTPUTS): print("Iterate b[] over the pivot: {}".format(pivot))
    if(b[which_row] != 0):
        b[which_row][0] = b[which_row][0]/pivot
        
    if(SHOW_OUTPUTS): print("Reseting a column over the pivot: {}".format(pivot))
    for i in range(len(a[which_row])):
        for j in range(len(a)):
            print("i = {}, j = {}".format(i,j))
            if(i != which_row and j != which_column):
                if(a[i][j] != 0):
                    value = (a[i][j]/pivot) * -1
                    if(SHOW_OUTPUTS): print("Using the value: {} for set 0 at the column".format(pivot))
                    a[i][j] += value
        
        
    return(c, a, b, certificate, operations, optimal)
    

def SimplexTableau(r, v, c, a, b):
    certificate = [0] * r
    operations = Identity(3)
    gap_vars = Identity(3)
    a = a + gap_vars
    c = c + [0] * r
    optimal = 0
    for i in range(len(c)):
        c[i] = c[i] * -1
        
    for c_index in range(len(c)):
        
        if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
        
        if(c[c_index] < 0 ):
            lower = 10000
            for i in range(len(b)):
                if(b[i][0] != 0):
                    if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(b[i][0],a[i][c_index]))
                    if(a[i][c_index] != 0 and b[i][0]/a[i][c_index] < lower):
                        lower = b[i][0]/a[i][c_index]
                        which_row = i
                        which_column = c_index
            if(SHOW_OUTPUTS): print("Lower value is {} on row #{}".format(lower, which_row))
            c, a, b, certificate, operations, optimal = Pivot(which_row, which_column, c, a, b, certificate, operations, optimal)

        

In [42]:
r = 3
v = 3
c = [2,4,8]
a = [[1,0,0],[0,1,0],[0,0,1]]
b = [[1],[1],[1]]
SimplexTableau(r,v,c,a,b)



c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
Optimal : 0


Minimizing: 1/1
Minimizing: 1/0
Minimizing: 1/0
Lower value is 1.0 on row #0
Pivot: 1
Iterate A[] over the pivot: 1
Iterate operations[] over the pivot: 1
Iterate b[] over the pivot: 1
Reseting a column over the pivot: 1
i = 0, j = 0
i = 0, j = 1
i = 0, j = 2
i = 0, j = 3
i = 0, j = 4
i = 0, j = 5
i = 1, j = 0
i = 1, j = 1
Using the value: 1 for set 0 at the column
i = 1, j = 2
i = 1, j = 3


IndexError: list index out of range

In [26]:
print(range(len(a)))

range(0, 3)
